In [5]:
pip install nbimporter

Note: you may need to restart the kernel to use updated packages.


In [6]:
import nbimporter

In [7]:
from database_queries import list_airbnb, sql_airbnb, mongo_airbnb
from airbnb_sorting_queries import sql_sort

SyntaxError: invalid syntax (<unknown>, line 1)

In [8]:
import psycopg, os
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np
import json
import csv
import datetime
from geopy.distance import distance

In [9]:
from flask import Flask, render_template, request
from datetime import datetime, date
import pandas as pd
import requests
import json

app = Flask("Airbnb_project")

In [10]:
events = []
airbnbs = []
event_id = ""
city_name = ""
event_date = date(2023,4,25)

@app.route('/')
def home():
    return render_template('concert_city_ver2.html')

@app.route('/city', methods=['POST'])
def city():
    #display selected city name in heading
    global city_name
    city_name = request.form['city_button']
    
    global event_date
    #get today's date
    event_date = date.today()
    
    
    #Retrive event info from API
    url = 'https://api.seatgeek.com/2/events'
    params = {
        'client_id': 'MjgxMjYwMTZ8MTY4MjEyODc5OC43OTA3NjUz',
        'sort': 'datetime_utc.asc',
        'per_page': 50,  
        'venue.city': city_name,
        'datetime_local.gte':event_date
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    #Format the data
    global events
    events = data['events']
    event_list = list()
    venue_list = list()
    date_list = list()
    id_list = list()
    for x in events:
        venue_list.append(x['venue']['name']) 
        event_list.append(x['title'])
        date_format = "%Y-%m-%dT%H:%M:%S"
        date_list.append((datetime.strptime(x['datetime_local'], date_format)).strftime("%Y-%m-%d %H:%M"))
        id_list.append(x['id'])
    
    return render_template('concert_event_ver3.html', 
                           city_name=city_name, 
                           event_list=event_list,
                           venue_list=venue_list,
                           date_list=date_list,
                           id_list=id_list,
                           date=event_date,
                           zip = zip)

@app.route('/filter_date', methods=['POST'])
def filter_date():
    #display selected city name in heading
    global city_name
    
    #get Date use date to search from API to get event list
    event_date = request.form['date']    
    
    #Retrive event info from API
    url = 'https://api.seatgeek.com/2/events'
    params = {
        'client_id': 'MjgxMjYwMTZ8MTY4MjEyODc5OC43OTA3NjUz',
        'sort': 'datetime_utc.asc',
        'per_page': 50,  
        'venue.city': city_name,
        'datetime_local.gte':event_date
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    #Format the data
    global events
    events = data['events']
    event_list = list()
    venue_list = list()
    date_list = list()
    id_list = list()
    for x in events:
        venue_list.append(x['venue']['name']) 
        event_list.append(x['title'])
        date_format = "%Y-%m-%dT%H:%M:%S"
        date_list.append((datetime.strptime(x['datetime_local'], date_format)).strftime("%Y-%m-%d %H:%M"))
        id_list.append(x['id'])
    
    return render_template('concert_event_ver3.html', 
                           city_name=city_name, 
                           event_list=event_list,
                           venue_list=venue_list,
                           date_list=date_list,
                           id_list=id_list,
                           date=event_date,
                           zip = zip)


@app.route('/event_list', methods=['POST'])
def event_list():
    #get selected event id from previous page
    global event_id
    event_id = int(request.form['event_id'])
    #get the informaiton of seleted event
    global events
    for x in events:
        if x['id'] == event_id:
            event = x['title']
            city = x['venue']['city']
            venue_name = x['venue']['name']
            venue_location = x['venue']['address']
            date_format = "%Y-%m-%dT%H:%M:%S"
            event_date = datetime.strptime(x['datetime_local'], date_format)
            ticket_link = x['performers'][0]['url']
            location_dict = x['venue']['location']
            
    global airbnbs
    geo_loc = (location_dict['lat'],location_dict['lon'])
    id_list = list_airbnb(geo_loc,1)
    airbnbs = sql_airbnb(id_list)
    
    name_list = list()
    property_type_list = list()
    room_type_list = list()
    accomodation_list = list()
    bedroom_list = list()
    bathroom_list = list()
    price_list = list()
    id_list = list()
    for x in airbnbs:
        name_list.append(x['name'])
        property_type_list.append(x['property_type'])
        room_type_list.append(x['room_type'])
        accomodation_list.append(x['accommodates'])
        bedroom_list.append(x['bathrooms'])
        bathroom_list.append(x['bedrooms'])
        price_list.append(x['price'])
        id_list.append(x['listing_id'])
    
    return render_template('sort_ver3.html', 
                           event=event,
                           city=city,
                           venue_name=venue_name,
                           venue_location=venue_location,
                           event_date=event_date,
                           ticket_link=ticket_link,
                           name_list=name_list,
                           property_type_list=property_type_list,
                           room_type_list=room_type_list,
                           accomodation_list=accomodation_list,
                           bedroom_list=bedroom_list,
                           bathroom_list=bathroom_list,
                           price_list=price_list,
                           id_list=id_list,
                           sort_name = "Price Descending",
                           zip=zip
                           )

@app.route('/sort', methods=['POST'])
def sort():
    sort = request.form['sort_listing']
    #sort 4 种：默认是price_dec，可以选price_asc,distance_dec,distance_asc
    
    if sort == "price_dec":
        sort_name = "Price Descending"
    elif sort == "price_asc":
        sort_name = "Price Ascending"
    
    global event_id
    global events
    for x in events:
        if x['id'] == event_id:
            event = x['title']
            city = x['venue']['city']
            venue_name = x['venue']['name']
            venue_location = x['venue']['address']
            date_format = "%Y-%m-%dT%H:%M:%S"
            event_date = datetime.strptime(x['datetime_local'], date_format)
            ticket_link = x['performers'][0]['url']
            location_dict = x['venue']['location']
    
    global airbnbs
    geo_loc = (location_dict['lat'],location_dict['lon'])
    id_list = list_airbnb(geo_loc,1)
    airbnbs = sql_sort(id_list, sort_name)
    
    name_list = list()
    property_type_list = list()
    room_type_list = list()
    accomodation_list = list()
    bedroom_list = list()
    bathroom_list = list()
    price_list = list()
    id_list = list()
    for x in airbnbs:
        if x['price'] != 0:
            name_list.append(x['name'])
            property_type_list.append(x['property_type'])
            room_type_list.append(x['room_type'])
            accomodation_list.append(x['accommodates'])
            bedroom_list.append(x['bathrooms'])
            bathroom_list.append(x['bedrooms'])
            price_list.append(x['price'])
            id_list.append(x['listing_id'])
        
    return render_template('sort_ver3.html', 
                           event=event,
                           city=city,
                           venue_name=venue_name,
                           venue_location=venue_location,
                           event_date=event_date,
                           ticket_link=ticket_link,
                           name_list=name_list,
                           property_type_list=property_type_list,
                           room_type_list=room_type_list,
                           accomodation_list=accomodation_list,
                           bedroom_list=bedroom_list,
                           bathroom_list=bathroom_list,
                           price_list=price_list,
                           id_list=id_list,
                           sort_name=sort_name,
                           zip=zip
                           )


@app.route('/airbnb', methods=['POST'])
def airbnb():
    global airbnbs
    airbnb_id = int(request.form['airbnb_id'])
    for x in airbnbs:
        if x['listing_id'] == airbnb_id:
            name = x['name']
            property_type = x['property_type']
            room_type = x['room_type']
            accommodates = x['accommodates']
            bathroom = x['bathrooms']
            bedroom = x['bedrooms']
            price = x['price']
            #可以加
            #min-nights
            #Revew Score
    
    #根据x['id']连mongodb调取comment
    comments = mongo_airbnb(airbnb_id)
    
    review_list = list()
    date_list = list()
    for x in comments[0]['reviews']:
        review_list.append(x['comments'])
        date_list.append(x['date'])
            
    return render_template('concert_airbnb_ver2.html', 
                           name=name,
                           property_type=property_type,
                           room_type=room_type,
                           accommodates=accommodates,
                           bathroom=bathroom,
                           bedroom=bedroom,
                           price=price,
                           review_list=review_list,
                           date_list=date_list,
                           sort=sort,
                           zip=zip
                           )

In [ ]:
app.run(host='localhost', port=5018)

 * Serving Flask app "Airbnb_project" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5018/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Jun/2023 16:22:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 16:22:31] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Jun/2023 16:22:33] "POST /city HTTP/1.1" 200 -
[2023-06-01 16:22:36,554] ERROR in app: Exception on /event_list [POST]
Traceback (most recent call last):
  File "/Users/cathy/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/cathy/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/cathy/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/cathy/opt/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/cathy/opt/anaconda3/lib/python3.9/site-packages/flask/a